In [1]:
import pandas as pd
import numpy as np

from ortools.graph import pywrapgraph

### read data

In [2]:
%%time
wish = pd.read_csv('/group/amfs_saving_model/jli/santa/child_wishlist_v2.csv', header=None).as_matrix()[:, 1:]
gift = pd.read_csv('/group/amfs_saving_model/jli/santa/gift_goodkids_v2.csv', header=None).as_matrix()[:, 1:]

CPU times: user 8.82 s, sys: 746 ms, total: 9.57 s
Wall time: 9.56 s


In [3]:
print wish.shape
print gift.shape

(1000000, 100)
(1000, 1000)


### get overall happiness

#### get child happy

In [4]:
child_happy_matrix = np.zeros((wish.shape[0], gift.shape[0]))
child_happy_row = [2000 * (wish.shape[1] - gift_rank) for gift_rank in range(wish.shape[1])]

In [5]:
%%time
for child_idx in range(wish.shape[0]):
    child_happy_matrix[child_idx] = -1000
    child_happy_matrix[child_idx][wish[child_idx]] = child_happy_row

CPU times: user 5.63 s, sys: 935 ms, total: 6.56 s
Wall time: 6.56 s


In [6]:
child_happy_matrix

array([[  -1000.,   -1000.,   -1000., ...,   40000.,   -1000.,   -1000.],
       [ 120000.,   -1000.,   -1000., ...,   -1000.,   -1000.,   -1000.],
       [  -1000.,   -1000.,   -1000., ...,   -1000.,   -1000.,   -1000.],
       ..., 
       [  -1000.,   -1000.,   -1000., ...,   -1000.,   -1000.,   -1000.],
       [  -1000.,   -1000.,   -1000., ...,   -1000.,   -1000.,   -1000.],
       [  -1000.,   -1000.,   -1000., ...,   -1000.,   -1000.,   -1000.]])

#### get gift happy

In [12]:
gift_happy_matrix = np.zeros((wish.shape[0], gift.shape[0]))
gift_happy_row = [2 * (gift.shape[1] - child_rank) for child_rank in range(gift.shape[1])]

In [13]:
%%time
for gift_idx in range(gift.shape[0]):
    gift_happy_matrix[:, gift_idx] = -1
    gift_happy_matrix[gift[gift_idx], gift_idx] = gift_happy_row

CPU times: user 24 s, sys: 1.55 s, total: 25.6 s
Wall time: 25.6 s


In [14]:
gift_happy_matrix

array([[-1., -1., -1., ..., -1., -1., -1.],
       [-1., -1., -1., ..., -1., -1., -1.],
       [-1., -1., -1., ..., -1., -1., -1.],
       ..., 
       [-1., -1., -1., ..., -1., -1., -1.],
       [-1., -1., -1., ..., -1., -1., -1.],
       [-1., -1., -1., ..., -1., -1., -1.]])

### sum together the happiness

In [19]:
%time all_happy = child_happy_matrix + gift_happy_matrix

CPU times: user 1.64 s, sys: 2.94 s, total: 4.59 s
Wall time: 4.59 s


In [20]:
all_happy

array([[  -1001.,   -1001.,   -1001., ...,   39999.,   -1001.,   -1001.],
       [ 119999.,   -1001.,   -1001., ...,   -1001.,   -1001.,   -1001.],
       [  -1001.,   -1001.,   -1001., ...,   -1001.,   -1001.,   -1001.],
       ..., 
       [  -1001.,   -1001.,   -1001., ...,   -1001.,   -1001.,   -1001.],
       [  -1001.,   -1001.,   -1001., ...,   -1001.,   -1001.,   -1001.],
       [  -1001.,   -1001.,   -1001., ...,   -1001.,   -1001.,   -1001.]])

### adjust for triplets and twins

In [28]:
for child_idx in range(0, 5001, 3):
    all_happy[range(child_idx, child_idx+3)] = np.mean(all_happy[range(child_idx, child_idx+3)], axis=0)
    
for child_idx in range(5001, 45001, 2):
    all_happy[range(child_idx, child_idx+2)] = np.mean(all_happy[range(child_idx, child_idx+2)], axis=0)

In [30]:
all_happy = all_happy.astype(np.int64)

In [31]:
all_happy

array([[39332, -1001, -1001, ..., 12665, -1001, -1001],
       [39332, -1001, -1001, ..., 12665, -1001, -1001],
       [39332, -1001, -1001, ..., 12665, -1001, -1001],
       ..., 
       [-1001, -1001, -1001, ..., -1001, -1001, -1001],
       [-1001, -1001, -1001, ..., -1001, -1001, -1001],
       [-1001, -1001, -1001, ..., -1001, -1001, -1001]])

## construct graph for ortools

In [34]:
min_cost_flow = pywrapgraph.SimpleMinCostFlow()

In [35]:
%%time
for child_idx in range(all_happy.shape[0]):
    
    if (child_idx % 50000 == 0) and (child_idx != 0):
        print('Finish %d children...' %(child_idx))
        
    for gift_idx in range(all_happy.shape[1]):
        gift_idx_true = gift_idx + all_happy.shape[0]
        min_cost_flow.AddArcWithCapacityAndUnitCost(child_idx, gift_idx_true, 1, -all_happy[child_idx, gift_idx])

Finish 50000 children...
Finish 100000 children...
Finish 150000 children...
Finish 200000 children...
Finish 250000 children...
Finish 300000 children...
Finish 350000 children...
Finish 400000 children...
Finish 450000 children...
Finish 500000 children...
Finish 550000 children...
Finish 600000 children...
Finish 650000 children...
Finish 700000 children...
Finish 750000 children...
Finish 800000 children...
Finish 850000 children...
Finish 900000 children...
Finish 950000 children...
CPU times: user 12min 54s, sys: 18.1 s, total: 13min 13s
Wall time: 13min 12s


In [36]:
supplies = [1] * all_happy.shape[0] + [-1000] * all_happy.shape[1]

In [37]:
len(supplies)

1001000

In [38]:
%%time
for i in range(len(supplies)):
    min_cost_flow.SetNodeSupply(i, supplies[i])

CPU times: user 438 ms, sys: 0 ns, total: 438 ms
Wall time: 437 ms


In [39]:
%%time 
if min_cost_flow.SolveMaxFlowWithMinCost() == min_cost_flow.OPTIMAL:
    print('Minimum cost:', min_cost_flow.OptimalCost())
else:
    print('There was an issue with the min cost flow input.')

('Minimum cost:', -195660229354)
CPU times: user 22min 21s, sys: 30.7 s, total: 22min 52s
Wall time: 22min 52s


### get the allocation

In [97]:
# list to record the assigned child
answ = np.zeros(len(wish), dtype=np.int32)
answ[:] = -1

In [98]:
%%time
for i in range(min_cost_flow.NumArcs()):
    if min_cost_flow.Flow(i) != 0:
        answ[min_cost_flow.Tail(i)] = min_cost_flow.Head(i) - wish.shape[0]

CPU times: user 6min 24s, sys: 1.53 s, total: 6min 25s
Wall time: 6min 25s


### adjust triplets

In [84]:
from collections import Counter

In [99]:
for t1 in range(0, 5001, 3):
    ans = dict(Counter(answ[range(t1, t1+3)]))
    
    if len(ans.keys()) == 1:
        continue
    elif len(ans.keys()) == 2:
        target_gift = [k for k in ans.keys() if ans[k]==2][0]
        can_kids = np.array([k_idx for k_idx in range(5001, wish.shape[0]) if answ[k_idx] == target_gift])
        can_scores = np.array([all_happy[k_idx, target_gift] for k_idx in can_kids])
        can_kid = can_kids[np.argmin(can_scores)]
        
        target_kid = [k for k in range(t1, t1+3) if answ[k]!=target_gift][0]
        answ[can_kid] = answ[target_kid]
        answ[target_kid] = target_gift
    else:
        t_replaces = []
        for t in range(t1, t1+3):
            replaces = np.array([other_idx for other_idx in range(5001, wish.shape[0]) if answ[other_idx] == answ[t]])
            replaces_scores = np.array([all_happy[other_idx, answ[t]] for other_idx in replaces])
            replace_two_idx = np.argsort(replaces_scores)[:2]
            replaces_two = replaces[replace_two_idx]
            t_replaces.append([t, replaces_two[0], replaces_two[1], np.mean(replaces_scores[replace_two_idx])])

        t_sorted = sorted(t_replaces, key=lambda x: x[3])
        answ[t_sorted[0][1]] = answ[t_sorted[1][0]]
        answ[t_sorted[0][2]] = answ[t_sorted[2][0]]
        answ[t_sorted[1][0]] = answ[t_sorted[0][0]]
        answ[t_sorted[2][0]] = answ[t_sorted[0][0]]

### adjust twins

In [100]:
%%time
for t1 in range(5001, 45001, 2):
    if answ[t1] == answ[t1+1]:
        continue
    else:
        t_replaces = []
        for t in [t1, t1+1]:
            replaces = np.array([other_idx for other_idx in range(45001, wish.shape[0]) if answ[other_idx] == answ[t]])
            replaces_scores = np.array([all_happy[other_idx, answ[t]] for other_idx in replaces])
            replaces_kid = replaces[np.argmin(replaces_scores)]
            t_replaces.append([t, replaces_kid, np.min(replaces_scores)])

        t_sorted = sorted(t_replaces, key=lambda x: x[2])
        answ[t_sorted[0][1]] = answ[t_sorted[1][0]]
        answ[t_sorted[1][0]] = answ[t_sorted[0][0]]

CPU times: user 48.5 s, sys: 3 ms, total: 48.5 s
Wall time: 48.5 s


### evaluation

In [95]:
import math
from fractions import gcd

def lcm(a, b):
    """Compute the lowest common multiple of a and b"""
    # in case of large numbers, using floor division
    return a * b // gcd(a, b)

def avg_normalized_happiness(pred, gift, wish):
    
    n_children = 1000000 # n children to give
    n_gift_type = 1000 # n types of gifts available
    n_gift_quantity = 1000 # each type of gifts are limited to this quantity
    n_gift_pref = 100 # number of gifts a child ranks
    n_child_pref = 1000 # number of children a gift ranks
    twins = math.ceil(0.04 * n_children / 2.) * 2    # 4% of all population, rounded to the closest number
    triplets = math.ceil(0.005 * n_children / 3.) * 3    # 0.5% of all population, rounded to the closest number
    ratio_gift_happiness = 2
    ratio_child_happiness = 2

    # check if triplets have the same gift
    for t1 in np.arange(0, triplets, 3):
        t1 = int(t1)
        triplet1 = pred[t1]
        triplet2 = pred[t1+1]
        triplet3 = pred[t1+2]
        # print(t1, triplet1, triplet2, triplet3)
        assert triplet1 == triplet2 and triplet2 == triplet3
                
    # check if twins have the same gift
    for t1 in np.arange(triplets, triplets+twins, 2):
        t1 = int(t1)
        twin1 = pred[t1]
        twin2 = pred[t1+1]
        # print(t1)
        assert twin1 == twin2

    max_child_happiness = n_gift_pref * ratio_child_happiness
    max_gift_happiness = n_child_pref * ratio_gift_happiness
    total_child_happiness = 0
    total_gift_happiness = np.zeros(n_gift_type)
    
    for i in range(len(pred)):
        child_id = i
        gift_id = pred[i]
        
        # check if child_id and gift_id exist
        assert child_id < n_children
        assert gift_id < n_gift_type
        assert child_id >= 0 
        assert gift_id >= 0
        child_happiness = (n_gift_pref - np.where(wish[child_id]==gift_id)[0]) * ratio_child_happiness
        if not child_happiness:
            child_happiness = -1

        gift_happiness = ( n_child_pref - np.where(gift[gift_id]==child_id)[0]) * ratio_gift_happiness
        if not gift_happiness:
            gift_happiness = -1

        total_child_happiness += child_happiness
        total_gift_happiness[gift_id] += gift_happiness
        
    denominator1 = n_children*max_child_happiness
    denominator2 = n_gift_quantity*max_gift_happiness*n_gift_type
    common_denom = lcm(denominator1, denominator2)
    multiplier = common_denom / denominator1

    ret = float(math.pow(total_child_happiness*multiplier,3) + \
        math.pow(np.sum(total_gift_happiness),3)) / float(math.pow(common_denom,3))
    return ret

In [101]:
%%time
score = avg_normalized_happiness(answ, gift, wish)
print('Predicted score: {:.8f}'.format(score))

Predicted score: 0.93609049
CPU times: user 18.5 s, sys: 2 ms, total: 18.5 s
Wall time: 18.5 s
